In [1]:
import re
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from nltk.corpus import wordnet as wn
from utils.wordnet_embeddings import closure_graph_bottomup
import itertools
import logging
import operator
import numpy as np
import glob
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

In [2]:
def create_graph(dict_synset, target_file):
    # merge all concepts into one list
    concept_synet = itertools.chain(*dict_synsets.values())
    # create graph
    g_bottomup = closure_graph_bottomup(concept_synet,
                      lambda s: s.hypernyms())

    # check for missing links in graph and merge all into root node
    missing_nodes = [node for node, in_degree in g_bottomup.in_degree() if in_degree == 0]
    g_bottomup.add_node('root')
    for i in missing_nodes:
        g_bottomup.add_edge( 'root', i)

    #missing_nodes = [node for node, in_degree in g_bottomup.in_degree() if in_degree == 0]
    nx.write_edgelist(g_bottomup , target_file,  delimiter=',', data=False, encoding='utf-8')
    df = pd.read_csv(target_file, names=['parent', 'child'])
    df.to_csv(target_file, index=False)
    




    
    

In [3]:
### retrieve all words from all datasets
data_files = glob.glob("datasets/*.csv")
#data_files = ["datasets/NOUN_mc-30.csv"]
word_list = []
column_names = ['word1', 'word2', 'sim']
for file in data_files:
    df = pd.read_csv(file) #, sep=';', names = column_names) #
    word_list.append(df['word1'].unique())
    word_list.append(df['word2'].unique())
    
word_list = list(set([item for sublist in word_list for item in sublist]))
dict_synsets = dict()
# load the benchmark datasets
# retrieve wordnet hierarchy for all words
wn_lemma = WordNetLemmatizer()

target_file = './Graph/'
for c in word_list:
    ls= []
    count = 0
    word1 = wn_lemma.lemmatize(c)
    for result in wn.synsets(word1 ): # retrieve all concepts related to this word
        ls.append(result)
    if ls != []:
        dict_synsets[c] = ls
        
create_graph (dict_synsets, target_file+'graph_wordsimilarity.csv')